In [ ]:
# Install deps
!pip install requests beautifulsoup4 pandas lxml openpyxl tqdm

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from urllib.parse import urljoin
from requests.adapters import HTTPAdapter, Retry
from datetime import datetime
from IPython.display import display
from tqdm import tqdm

# --- Config ---
BASE = "https://www.goodreads.com"
SEARCH_BASE = "https://www.goodreads.com/search"
QUERIES = [
    "data science",
    "machine learning",
    "artificial intelligence",
    "business",
    "history",
    "psychology",
]  # add or replace categories here
MAX_PAGES_PER_QUERY = 5
DELAY = 1.2  # between search pages
DETAIL_DELAY = 1.0  # between fetching individual book pages
TIMEOUT = 15

# Session with retry
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.3, status_forcelist=[500,502,503,504])
session.mount("https://", HTTPAdapter(max_retries=retries))
session.headers.update({"User-Agent": "Mozilla/5.0 (compatible; goodreads-multi-scraper/1.0)"})

def clean_string(s: str) -> str:
    return " ".join(s.strip().split()) if s else s

def parse_search_page(html, category_label):
    soup = BeautifulSoup(html, "lxml")
    results = []
    rows = soup.select("table.tableList tr")
    for row in rows:
        try:
            title_tag = row.select_one("a.bookTitle")
            title = clean_string(title_tag.get_text()) if title_tag else None
            book_url = urljoin(BASE, title_tag["href"]) if title_tag and title_tag.has_attr("href") else None

            author_tag = row.select_one("a.authorName")
            author = clean_string(author_tag.get_text()) if author_tag else None

            rating_tag = row.select_one("span.minirating")
            average_rating = None
            num_ratings = None
            if rating_tag:
                text = rating_tag.get_text()
                m_avg = re.search(r"([0-9]+\.[0-9]+) avg rating", text)
                m_count = re.search(r"—\s*([\d,]+)\s*ratings", text)
                if m_avg:
                    average_rating = m_avg.group(1)
                if m_count:
                    num_ratings = m_count.group(1).replace(",", "")
            results.append({
                "category": category_label,
                "title": title,
                "author": author,
                "average_rating": average_rating,
                "num_ratings": num_ratings,
                "book_url": book_url
            })
        except Exception as e:
            print(f"Parsing row failed: {e}")
            continue
    return results

def scrape_goodreads_for_query(query, max_pages=3):
    all_books = []
    for page in range(1, max_pages + 1):
        params = {"q": query, "search_type": "books", "page": page}
        print(f"[{query} - page {page}]")
        try:
            resp = session.get(SEARCH_BASE, params=params, timeout=TIMEOUT)
            if resp.status_code != 200:
                print(f"  -> HTTP {resp.status_code}, stopping {query}.")
                break
            page_results = parse_search_page(resp.text, category_label=query)
            if not page_results:
                print(f"  -> No results parsed for {query} page {page}.")
                break
            all_books.extend(page_results)
        except Exception as e:
            print(f"  -> Request failed for {query} page {page}: {e}")
        time.sleep(DELAY)
    df = pd.DataFrame(all_books)
    df = df.drop_duplicates(subset=["category", "title", "author"]).reset_index(drop=True)
    return df

def extract_work_id(book_url):
    """Fetch book detail page and extract the work ID from '/work/<id>' in HTML."""
    try:
        resp = session.get(book_url, timeout=TIMEOUT)
        if resp.status_code != 200:
            return None
        text = resp.text
        # Attempt to find '/work/<number>' pattern
        m = re.search(r'/work/(\d+)', text)
        if m:
            return m.group(1)
        # Fallback: canonical link
        canon = re.search(r'<link rel="canonical" href="([^"]+)"', text)
        if canon:
            m2 = re.search(r'/work/(\d+)', canon.group(1))
            if m2:
                return m2.group(1)
        return None
    except Exception:
        return None

# Scrape all categories
dfs = []
for q in QUERIES:
    df_q = scrape_goodreads_for_query(q, max_pages=MAX_PAGES_PER_QUERY)
    print(f"Collected {len(df_q)} rows for category '{q}'")
    dfs.append(df_q)

combined = pd.concat(dfs, ignore_index=True)

# Fetch work_id for each unique book_url (to avoid redundant requests)
combined["work_id"] = None
unique_urls = combined["book_url"].dropna().unique()
work_id_map = {}

print("\nResolving work IDs (this will take a bit)...")
for url in tqdm(unique_urls, desc="work_id fetch"):
    work_id = extract_work_id(url)
    work_id_map[url] = work_id
    time.sleep(DETAIL_DELAY)  # polite delay between detail page requests

# Map back
combined["work_id"] = combined["book_url"].map(work_id_map)

# Save all into one Excel workbook: each category sheet + combined sheet
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
excel_filename = f"goodreads_multi_with_workid_{timestamp}.xlsx"
with pd.ExcelWriter(excel_filename, engine="openpyxl") as writer:
    combined.to_excel(writer, sheet_name="Combined", index=False)
    for df_q in dfs:
        if df_q.empty:
            continue
        # Attach work_id to category-specific sheet
        cat = df_q["category"].iloc[0]
        df_cat = combined[combined["category"] == cat].drop_duplicates(subset=["title", "author", "work_id"])
        sheet_name = cat[:30]
        df_cat.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\nSaved Excel workbook to {excel_filename} with work IDs included.")
display(combined.head(10))


[data science - page 1]
[data science - page 2]
[data science - page 3]
[data science - page 4]
[data science - page 5]
Collected 49 rows for category 'data science'
[machine learning - page 1]
[machine learning - page 2]
[machine learning - page 3]
[machine learning - page 4]
[machine learning - page 5]
Collected 48 rows for category 'machine learning'
[artificial intelligence - page 1]
[artificial intelligence - page 2]
[artificial intelligence - page 3]
[artificial intelligence - page 4]
[artificial intelligence - page 5]
Collected 48 rows for category 'artificial intelligence'
[business - page 1]
[business - page 2]
[business - page 3]
[business - page 4]
[business - page 5]
Collected 48 rows for category 'business'
[history - page 1]
[history - page 2]
[history - page 3]
[history - page 4]
[history - page 5]
Collected 50 rows for category 'history'
[psychology - page 1]
[psychology - page 2]
[psychology - page 3]
[psychology - page 4]
[psychology - page 5]
Collected 49 rows for ca

work_id fetch: 100%|██████████| 292/292 [11:21<00:00,  2.33s/it]



Saved Excel workbook to goodreads_multi_with_workid_20250731_152404.xlsx with work IDs included.


,category,title,author,average_rating,num_ratings,book_url,work_id
0,data science,Data Science for Business: What You Need to Kn...,Foster Provost,4.13,2588,https://www.goodreads.com/book/show/17912916-d...,25100684
1,data science,Data Smart: Using Data Science to Transform In...,John W. Foreman,4.12,1012,https://www.goodreads.com/book/show/17682206-d...,24982572
2,data science,Data Science from Scratch: First Principles wi...,Joel Grus,3.91,1118,https://www.goodreads.com/book/show/25407018-d...,45161385
3,data science,Mindmasters: The Data-Driven Science of Predic...,Sandra Matz,4.10,239,https://www.goodreads.com/book/show/205905685-...,211953591
4,data science,"R for Data Science: Import, Tidy, Transform, V...",Hadley Wickham,4.54,1189,https://www.goodreads.com/book/show/33399049-r...,54149357
5,data science,Doing Data Science: Straight Talk from the Fro...,Cathy O'Neil,3.74,570,https://www.goodreads.com/book/show/17346997-d...,24087179
6,data science,Machine Learning For Absolute Beginners: A Pla...,Oliver Theobald,4.11,457,https://www.goodreads.com/book/show/35518108-m...,56933474
7,data science,Numsense! Data Science for the Layman: No Math...,Annalyn Ng,4.15,608,https://www.goodreads.com/book/show/34213247-n...,55261521
8,data science,Python Data Science Handbook: Essential Tools ...,Jake VanderPlas,4.29,651,https://www.goodreads.com/book/show/26457146-p...,46459730
9,data science,Data Science (The MIT Press Essential Knowledg...,John D. Kelleher,3.91,840,https://www.goodreads.com/book/show/36722689-d...,58519365
